In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd, numpy as np

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [12]:
X_train = torch.tensor(pd.read_csv("./data/x_train.csv").to_numpy(), dtype=float)
y_train = torch.tensor(pd.read_csv("./data/y_train.csv").to_numpy(), dtype=float)

X_test = torch.tensor(pd.read_csv("./data/x_test.csv").to_numpy(), dtype=float)
y_test = torch.tensor(pd.read_csv("./data/y_test.csv").to_numpy(), dtype=float)

In [13]:
# -----------------------------
# INITIALIZE MODEL, LOSS, AND OPTIMIZER
# -----------------------------

# Create an instance of your model (a simple linear regression model here)
# This model will learn a relationship like: y = w*x + b
model = nn.Linear(in_features=X_train.shape[1], out_features=1, dtype=float)

# Define the loss function (how we measure the model’s error)
# nn.MSELoss() = Mean Squared Error → average of squared differences between predicted and actual values
criterion = nn.MSELoss()

# Define the optimizer — this updates model weights to reduce the loss
# SGD = Stochastic Gradient Descent, a common optimization algorithm
# model.parameters() = tells optimizer which parameters to update (weights & biases)
# lr=0.01 = learning rate, controls how big a step we take each update
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [14]:
# -----------------------------
# TRAINING LOOP FOR A MODEL
# -----------------------------

# Set a small tolerance value — if the loss stops changing by more than this, we’ll stop training early
tol = 1e-5

# Set the maximum number of epochs (full passes through the dataset)
# 1e7 = 10 million (a very large number, used here as an upper limit)
epochs = int(1e7)

# Store the previous loss value; start with infinity so any real loss will be smaller
prev_loss = float("inf")

# Loop through each epoch (training iteration)
for epoch in range(epochs):

    # ---- Forward pass ----
    # Feed the training data (X_train) to the model to get predictions (outputs)
    outputs = model(X_train)

    # Calculate how far off the predictions are from the true labels (y_train)
    # The 'criterion' defines the loss function (e.g., MSE, cross-entropy, etc.)
    loss = criterion(outputs, y_train)

    # ---- Backward pass & optimization ----
    # Clear (reset) any gradients from the previous step
    optimizer.zero_grad()

    # Compute new gradients for each model parameter based on the current loss
    loss.backward()

    # Update model parameters (weights and biases) using the optimizer
    optimizer.step()

    # ---- Check progress ----
    # Convert the loss (a PyTorch tensor) to a plain Python number
    curr_loss = loss.item()

    # Print the current epoch number and loss every 50 epochs to track progress
    if (epoch + 1) % 50 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {curr_loss:.4f}")

    # ---- Early stopping condition ----
    # If the loss hasn’t changed much compared to the last epoch, stop training
    if abs(curr_loss - prev_loss) <= tol:
        break

    # Save current loss as previous loss for the next iteration
    prev_loss = curr_loss

Epoch [50/10000000], Loss: 17355816724.9602
Epoch [100/10000000], Loss: 14824548919.5148
Epoch [150/10000000], Loss: 14054149917.3808
Epoch [200/10000000], Loss: 13645389786.8737
Epoch [250/10000000], Loss: 13359974405.7485
Epoch [300/10000000], Loss: 13136569941.2386
Epoch [350/10000000], Loss: 12953480423.7834
Epoch [400/10000000], Loss: 12799986047.1407
Epoch [450/10000000], Loss: 12669361071.6705
Epoch [500/10000000], Loss: 12556847663.5138
Epoch [550/10000000], Loss: 12458895451.0072
Epoch [600/10000000], Loss: 12372785457.3475
Epoch [650/10000000], Loss: 12296403635.8952
Epoch [700/10000000], Loss: 12228088939.2749
Epoch [750/10000000], Loss: 12166525477.4686
Epoch [800/10000000], Loss: 12110663450.4522
Epoch [850/10000000], Loss: 12059659950.5571
Epoch [900/10000000], Loss: 12012833988.1374
Epoch [950/10000000], Loss: 11969631958.8825
Epoch [1000/10000000], Loss: 11929600920.0461
Epoch [1050/10000000], Loss: 11892367787.8436
Epoch [1100/10000000], Loss: 11857623070.9941
Epoch [1

In [15]:
torch.save(model, "./model/model.pt")

In [16]:
model = torch.load("./model/model.pt", weights_only=False)

In [17]:
y_pred = model(X_test)

y_test_flat, y_pred_flat = y_test.numpy().flatten(), y_pred.detach().numpy().flatten()

mse = mean_squared_error(y_test_flat, y_pred_flat)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_flat, y_pred_flat)
r2 = r2_score(y_test_flat, y_pred_flat)

print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R2 Score: {r2:.4f}")

Mean Squared Error (MSE): 12297254356.0164
Root Mean Squared Error (RMSE): 110892.9861
Mean Absolute Error (MAE): 87413.1106
R2 Score: 0.5031
